# Deep Learning Assessment: Customer Churn Prediction

In this notebook, we'll apply deep learning concepts to predict customer churn - a critical business challenge. You'll learn how neural networks can identify patterns in customer behavior data to predict which customers are likely to leave.

## Learning Objectives
1. Understand how to structure data for deep learning
2. Build and train a neural network using PyTorch
3. Evaluate model performance on a business metric
4. Complete an assessment that tests your ability to improve model performance

## Assessment Criteria
To pass this assessment, your model must achieve:
- Test accuracy > 85% on the holdout set
- F1 score > 0.80 for the churn class

These metrics ensure your model is both accurate overall and good at identifying customers likely to churn.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

## Generate Synthetic Customer Data

We'll create a synthetic dataset that mimics real customer behavior data. Features include:
- Tenure (months)
- Monthly charges
- Total charges
- Number of support calls
- Service usage score

In [ ]:
def generate_customer_data(n_samples=1000):
    # Generate features
    tenure = np.random.randint(1, 72, n_samples)  # 1-72 months
    monthly_charges = np.random.uniform(30, 150, n_samples)  # $30-$150
    total_charges = monthly_charges * tenure + np.random.normal(0, 100, n_samples)
    support_calls = np.random.poisson(2, n_samples)  # Average 2 calls
    usage_score = np.random.uniform(0, 100, n_samples)  # 0-100 usage score
    
    # Combine features
    X = np.column_stack([tenure, monthly_charges, total_charges, support_calls, usage_score])
    
    # Generate churn labels based on a rule
    churn_score = (
        -0.1 * tenure +  # Longer tenure = less likely to churn
        0.3 * (monthly_charges / 50) +  # Higher charges = more likely to churn
        0.2 * (support_calls / 2) +  # More support calls = more likely to churn
        -0.4 * (usage_score / 50)  # Higher usage = less likely to churn
    )
    
    # Add some randomness
    churn_score += np.random.normal(0, 0.1, n_samples)
    
    # Convert to binary labels
    y = (churn_score > 0).astype(np.int64)
    
    return X, y

# Generate data
X, y = generate_customer_data()

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test)

# Print dataset information
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Churn rate in training set: {y_train.mean():.2%}")

## Visualize the Data

Let's examine how different features relate to churn probability. This helps us understand what patterns the neural network might learn.

In [ ]:
def plot_feature_relationships():
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    feature_names = ['Tenure', 'Monthly Charges', 'Support Calls', 'Usage Score']
    feature_indices = [0, 1, 3, 4]  # Skip total charges as it's derived from tenure
    
    for i, (name, idx) in enumerate(zip(feature_names, feature_indices)):
        churned = X_train[y_train == 1, idx]
        stayed = X_train[y_train == 0, idx]
        
        axes[i].hist([stayed, churned], bins=20, label=['Stayed', 'Churned'], alpha=0.6)
        axes[i].set_title(f'{name} Distribution by Churn Status')
        axes[i].set_xlabel(name)
        axes[i].set_ylabel('Count')
        axes[i].legend()
    
    plt.tight_layout()
    plt.show()

plot_feature_relationships()

## Define the Neural Network

We'll create a simple neural network with:
- Input layer (5 features)
- Two hidden layers
- Output layer (1 neuron for binary classification)

This architecture should be able to capture non-linear relationships in the data.

In [ ]:
class ChurnPredictor(nn.Module):
    def __init__(self, input_size=5):
        super(ChurnPredictor, self).__init__()
        self.layer1 = nn.Linear(input_size, 16)
        self.layer2 = nn.Linear(16, 8)
        self.layer3 = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.layer3(x))
        return x

# Initialize the model
model = ChurnPredictor()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

print("Model architecture:")
print(model)

## Train the Model

We'll train the model for several epochs and monitor both training loss and validation metrics.

In [ ]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=100):
    train_losses = []
    test_metrics = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train.view(-1, 1))
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        
        # Evaluation
        if (epoch + 1) % 10 == 0:
            model.eval()
            with torch.no_grad():
                test_outputs = model(X_test)
                test_preds = (test_outputs >= 0.5).float()
                accuracy = accuracy_score(y_test, test_preds)
                f1 = f1_score(y_test, test_preds)
                test_metrics.append((epoch, accuracy, f1))
                print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, '
                      f'Test Accuracy: {accuracy:.4f}, F1: {f1:.4f}')
    
    return train_losses, test_metrics

# Train the model
train_losses, test_metrics = train_model(model, X_train_tensor, y_train_tensor, 
                                        X_test_tensor, y_test_tensor)

## Visualize Training Progress

In [ ]:
def plot_training_progress(train_losses, test_metrics):
    epochs, accuracies, f1_scores = zip(*test_metrics)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot training loss
    ax1.plot(train_losses)
    ax1.set_title('Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    
    # Plot test metrics
    ax2.plot(epochs, accuracies, label='Accuracy')
    ax2.plot(epochs, f1_scores, label='F1 Score')
    ax2.set_title('Test Metrics')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Score')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

plot_training_progress(train_losses, test_metrics)

## Final Model Evaluation

In [ ]:
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_preds = (test_outputs >= 0.5).float()
        
        accuracy = accuracy_score(y_test, test_preds)
        f1 = f1_score(y_test, test_preds)
        
        print("\nFinal Model Performance:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print("\nDetailed Classification Report:")
        print(classification_report(y_test, test_preds))
        
        # Check if model passes assessment criteria
        passes_accuracy = accuracy > 0.85
        passes_f1 = f1 > 0.80
        
        print("\nAssessment Results:")
        print(f"Accuracy > 85%: {'✓' if passes_accuracy else '✗'}")
        print(f"F1 Score > 0.80: {'✓' if passes_f1 else '✗'}")
        print(f"Overall: {'PASS' if (passes_accuracy and passes_f1) else 'FAIL'}")
        
        return passes_accuracy and passes_f1

passed = evaluate_model(model, X_test_tensor, y_test_tensor)

## Your Assessment Task

If your model didn't pass both criteria (accuracy > 85% and F1 > 0.80), modify the model architecture or training process to achieve passing scores. You can:

1. Add more layers or neurons
2. Adjust the learning rate
3. Train for more epochs
4. Add dropout for regularization
5. Try different optimizers

Use the code cell below to implement your improvements:

In [ ]:
# Your improved model here
class ImprovedChurnPredictor(nn.Module):
    def __init__(self, input_size=5):
        super(ImprovedChurnPredictor, self).__init__()
        # TODO: Implement your improved architecture
        pass
    
    def forward(self, x):
        # TODO: Implement forward pass
        pass

# Uncomment and modify these lines to train your improved model
# improved_model = ImprovedChurnPredictor()
# criterion = nn.BCELoss()
# optimizer = optim.Adam(improved_model.parameters(), lr=0.01)
# train_losses, test_metrics = train_model(improved_model, X_train_tensor, y_train_tensor, 
#                                         X_test_tensor, y_test_tensor, epochs=100)
# plot_training_progress(train_losses, test_metrics)
# passed = evaluate_model(improved_model, X_test_tensor, y_test_tensor)